In [33]:
import os

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [34]:

from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = server = MCPServerSSE('http://localhost:8090/sse') 

In [36]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = OpenAIModel("Qwen3-8B", provider=provider)

In [ ]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [ ]:
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

output = {}
output["Request_id"] = data["Request_id"]
output["Datetime"] = data["Datetime"]
output["Location"] = data["Location"]
output["Request_id"] = data["Request_id"]
output["From"] = data["From"]
output["Location"] = data["Location"]
output["Attendees"] = data["Attendees"]
for i in range(len(output["Attendees"])):
    email = output["Attendees"]["email"]
    occupied = 


In [42]:
SYSTEM_PROMPT = """
        "You have access to:\n"
        "   1. get_current_time(params: dict)\n"
        "Use this tool for date/time questions."
"""

inp = """
{
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
"""

In [43]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model,
    toolsets=[server],
    system_prompt = SYSTEM_PROMPT
)

In [44]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output


In [49]:
import time
import asyncio

start = time.perf_counter()
result = await run_async("current date and day of the week")
end = time.perf_counter()

print(f"Result: {result}")
print(f"Elapsed time: {end - start:.2f} seconds")

Result: 

The current date is **July 19, 2025**, and it is a **Saturday**. 

Wait, there seems to be a discrepancy here. Earlier calculations suggested Thursday, but the actual day for July 19, 2025, is a **Saturday**. Let me correct that. 

Using a reliable source or calendar, July 19, 2025, falls on a **Saturday**. Therefore, the accurate response is:

**The current date is July 19, 2025, which is a Saturday.**
Elapsed time: 21.38 seconds


In [ ]:
VLLM_USE_TRITON_FLASH_ATTN=0 \
vllm serve Qwen/Qwen3-8B \
    --served-model-name Qwen3-8B \
    --api-key abc-123 \
    --port 8000 \
    --enable-auto-tool-choice \
    --tool-call-parser hermes \
    --trust-remote-code